In [ ]:
!pip uninstall -y tf-nightly keras-nightly tensorflow keras
!pip install tensorflow==2.16.2 keras==3.3.3

In [ ]:

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import os
train_df = pd.read_csv(train_file_path , sep='\t', header=None, names=["label", "message"])
#print(train_df)
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])
#print(test_df)


train_features = train_df["message"]
train_labels = train_df["label"]

test_features = test_df["message"]
test_labels = test_df["label"]


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_features)

train_seq = tokenizer.texts_to_sequences(train_features)
train_padded = pad_sequences(train_seq, maxlen=100, padding="post")

# Same for test set
test_seq = tokenizer.texts_to_sequences(test_features)
test_padded = pad_sequences(test_seq, maxlen=100, padding="post")

# Labels to 0/1
y_train = (train_labels == "spam").astype(int)
y_test = (test_labels == "spam").astype(int)



In [ ]:

model = keras.Sequential([
    keras.layers.Embedding(10000, 16, input_length=100),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])


model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

history = model.fit(
    train_padded, y_train,
    epochs=40,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = pad_sequences(seq, maxlen=100, padding='post')
    percentage = model.predict(padded, verbose=0)[0][0]

    if percentage > 0.5:
        return [float(percentage), 'spam']
    else:
        return [float(1-percentage), 'ham']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
